In [1]:
from scipy.spatial import Delaunay
from classes import Parallel
from pathlib import Path
from pymongo import MongoClient as mc
import numpy as np
import math

In [2]:
def basicFeatures(mpts):
    fv = []
    
    res = []
    res.append(mpts[0])
    mpts = mpts[1]
    mpts = [i[:2] for i in mpts]
    tri = Delaunay(mpts)
    triangles = tri.simplices
    for t in triangles:
        vertices = [mpts[i] for i in t]
        l1 = ((vertices[1][0]-vertices[2][0])**2+(vertices[1][1]-vertices[2][1])**2)**0.5
        l2 = ((vertices[0][0]-vertices[2][0])**2+(vertices[0][1]-vertices[2][1])**2)**0.5
        l3 = ((vertices[0][0]-vertices[1][0])**2+(vertices[0][1]-vertices[1][1])**2)**0.5
        alpha1 = math.acos((l2*l2+l3*l3-l1*l1)/(2*l2*l3))
        alpha2 = math.acos((l1*l1+l3*l3-l2*l2)/(2*l3*l1))
        alpha3 = math.acos((l2*l2+l1*l1-l3*l3)/(2*l2*l1))
        alpha_max = max(alpha1,max(alpha2,alpha3))
        if alpha_max>(2/3)*math.pi:
            continue
        alpha=math.cos(alpha_max)
        p=l1+l2+l3
        s=p/2
        a=(s*(s-l1)*(s-l2)*(s-l3))**0.5
        beta=(p*p)/a
        gamma=max(l1,max(l2,l3))/min(l1,min(l2,l3))
        fv.append([alpha, beta, gamma])
    res.append(fv)
    return res

In [3]:
db_database = mc('10.5.18.101').BI
print(type(db_database))
db_collection = db_database['Anguli_200k_1M']
print(type(db_collection))
all_records = db_collection.find()
print(all_records.count())

<class 'pymongo.database.Database'>
<class 'pymongo.collection.Collection'>
1200000


/tmp/ipykernel_40404/1852542332.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(all_records.count())


In [4]:
proc_set = [[i['_id'], i['mv']] for i in db_collection.find({})]


In [5]:
print(len(proc_set))
print(type(proc_set[0][0]))
l = 0
for i in proc_set:
    if l < 2:   
        print()
        l += 1

1200000
<class 'bson.objectid.ObjectId'>




Code for extracting all features

In [6]:
data = []
proc_list = proc_set
for i in proc_list[:5000]:
    data.append(basicFeatures(i))


In [7]:
vctrs = [i[1] for i in data]

In [8]:
kmns = []
for i in vctrs:
    kmns = kmns + i

In [9]:
len(kmns)

625705

In [10]:
kmns[:20]

[[0.20695859104439152, 23.239079073742147, 1.6363224072315832],
 [0.06507913734559692, 27.531965329832087, 2.2730003546512916],
 [0.09788826992209923, 22.46941536248438, 1.3636509148337823],
 [0.2819973081348389, 22.247360525108114, 1.4571842504438999],
 [-0.02558888781907636, 23.814807432387095, 1.573100451473447],
 [0.1318438591124176, 28.00974524810508, 2.339007826796387],
 [-0.056847423639598965, 25.363153716761595, 1.867503030136114],
 [0.0819155567294537, 23.044381242941455, 1.5452931872245987],
 [0.3454632411252103, 22.31619472951742, 1.4442188695835112],
 [0.06244649161166468, 27.285270496805058, 2.2375851559922233],
 [-0.3538920974158912, 28.548621110036883, 1.7542950209152859],
 [0.12835604651426002, 23.113272373756875, 1.5970255097957131],
 [0.2998369371092684, 22.498686330998524, 1.4951354087050213],
 [-0.041202096209968335, 28.659118429894367, 2.3934106297708624],
 [-0.0343332755538526, 24.535175043210483, 1.7355403364057171],
 [0.305064472298919, 21.894770844910138, 1.387

In [11]:
from scipy.cluster.vq import vq, kmeans, whiten

wtend = whiten(np.array(kmns))

In [12]:
wtend

array([[ 0.86566456,  1.62586052,  0.91904663],
       [ 0.27221244,  1.92620092,  1.27663919],
       [ 0.40944619,  1.57201304,  0.76589966],
       ...,
       [ 1.49897503,  1.48600655,  0.69263885],
       [ 0.00490362,  1.63907807,  0.8475555 ],
       [-0.04080578,  2.58313207,  1.9746468 ]])

In [13]:
points = np.array((wtend[0], wtend[1], wtend[2]))

In [14]:
fnl = kmeans(wtend, 300)

In [16]:
type(fnl)

tuple

In [18]:
len(fnl)

2

In [20]:
from sklearn.cluster import KMeans

k = KMeans(n_clusters= 300, random_state= 0).fit(np.array(kmns))

In [37]:
clus = [0 for i in range(301)]

In [22]:
k

KMeans(n_clusters=300, random_state=0)

In [26]:
len(k.cluster_centers_)

300

In [43]:
t = np.array(kmns)
ch = k.predict([t[2]])
k.predict([t[2]])

array([283], dtype=int32)

In [44]:
ch[0]

283

In [46]:
for i in t:
    idx = k.predict([i])
    clus[idx[0]] += 1

In [ ]:
from matplotlib.pyplot import plot as plt

xaxis = [i+1 for i in range(301)]